# Automated ML

In this notebook, I will perform the following steps:

Import Dependencies
Create an Experiment in an existing Workspace.
Create or Attach existing AmlCompute to a workspace.
Define data loading in a TabularDataset.
Configure AutoML using AutoMLConfig.
Use AutoMLStep
Train the model using AmlCompute
Explore the results.
Test the best fitted model.

## Import Dependcies

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [21]:
!pip install --upgrade --upgrade-strategy eager azureml-sdk[automl,widgets]
#Restart Kernel after update

/bin/bash: pip: command not found


In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
import json
import pickle

import azureml.core
from azureml.core import Model
from azureml.core.environment import Environment
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import Model
from azureml.core.webservice import LocalWebservice
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.core.model import Model

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

import urllib.request


import joblib



SDK version: 1.40.0


## Create an Azure ML experiment

In [2]:
ws = Workspace.from_config()

# Choose a name for the run history container in the workspace.
experiment_name = 'ml-experiment-1'
project_folder = './automl-project'

experiment = Experiment(ws, experiment_name)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-192511
aml-quickstarts-192511
southcentralus
9b72f9e6-56c5-4c16-991b-19c652994860


## Create or Attach an AmlCompute cluster

In [3]:
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automl-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded......................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Heart_Prediction_Dataset"
description_text = "Heart Prediction Dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/camille-wilkens/AZ_Capstone/main/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## Review the Dataset Result

In [5]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
     "n_cross_validations": 25,
     "primary_metric" : 'AUC_weighted'
}
 #   "primary_metric" : 'AUC_weighted'
# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)


Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
ml-experiment-1,AutoML_9ae81c6d-3fc1-45c9-b5c6-a211ba6b00f8,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [10]:

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [11]:

remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
ml-experiment-1,AutoML_9ae81c6d-3fc1-45c9-b5c6-a211ba6b00f8,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feat

{'runId': 'AutoML_9ae81c6d-3fc1-45c9-b5c6-a211ba6b00f8',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-04-18T12:30:14.048036Z',
 'endTimeUtc': '2022-04-18T12:48:57.2524Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '25',
  'target': 'automl-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"ml-experiment-1","subscription_id":"9b72f9e6-56c5-4c16-991b-19c652994860","resource_group":"aml-quickstarts-192511","workspace_name":"quick-starts-ws-192511","region":"southcentralus","compute_target":"autom

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
best_run,fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics() 

print("Best Run:",best_run.id)
print(best_run)
print("Fitted Model:", fitted_model)
print("Fitted Model Steps:")
print(fitted_model.steps)
print("Best Run Metrics")
print(best_run_metrics)

for i in best_run_metrics:
    x = best_run_metrics[i]
    print(i,x )


Best Run: AutoML_9ae81c6d-3fc1-45c9-b5c6-a211ba6b00f8_38
Run(Experiment: ml-experiment-1,
Id: AutoML_9ae81c6d-3fc1-45c9-b5c6-a211ba6b00f8_38,
Type: azureml.scriptrun,
Status: Completed)
Fitted Model: Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('16', Pipeline(memory=None, steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('extratreesclassifier', ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None, criterion='gini', max_depth=None, max_features=None, max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0

In [13]:
print(fitted_model.steps)

[('datatransformer', DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, task='classification')), ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('16', Pipeline(memory=None, steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('extratreesclassifier', ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None, criterion='gini', max_depth=None, max_features=None, max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.01, min_samples_split=0.056842105263157895, min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1, oob_score=False, random_state=None, verbose=0, warm_start=False))], verbose=False)), ('28', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy

In [14]:
automl_env=best_run.get_environment()
print(automl_env.python.conda_dependencies.serialize_to_string())


channels:
- anaconda
- conda-forge
- pytorch
dependencies:
- python=3.7.9
- pip=20.2.4
- pip:
  - azureml-core==1.40.0
  - azureml-mlflow==1.40.0
  - azureml-pipeline-core==1.40.0
  - azureml-telemetry==1.40.0
  - azureml-interpret==1.40.0
  - azureml-responsibleai==1.40.0
  - azureml-automl-core==1.40.0
  - azureml-automl-runtime==1.40.0
  - azureml-train-automl-client==1.40.0
  - azureml-train-automl-runtime==1.40.0.post1
  - azureml-dataset-runtime==1.40.0
  - azureml-defaults==1.40.0
  - inference-schema
  - py-cpuinfo==5.0.0
  - mpi4py==3.1.3
  - click<8.0.0
  - pystan==2.19.1.1
  - boto3==1.15.18
  - botocore==1.18.18
  - joblib==0.14.1
  - cloudpickle==1.6.0
- numpy~=1.18.0
- scikit-learn==0.22.1
- pandas~=1.1.5
- py-xgboost==1.3.3
- fbprophet==0.7.1
- holidays==0.10.3
- setuptools-git
- psutil>5.0.0,<6.0.0
name: azureml_61b883c981e1199704b60f5cc50378fa



## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

### Register Model

In [15]:
#Steps Followed from: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=azcli

from azureml.core import Model

automl_model = remote_run.register_model(model_name='Heart_Failure_Prediction_Model')
automl_model.download(target_dir="outputs", exist_ok = True)



'outputs/model.pkl'

In [16]:
for model in Model.list(ws):
    print(model.name,model.version)

Heart_Failure_Prediction_Model 1


### Create Inference Config

In [17]:

from azureml.automl.core.shared import constants
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

myenv = Environment.from_conda_specification(name="env", file_path="project_environment.yml")
inference_config = InferenceConfig(entry_script="score.py",environment=myenv)

TODO: In the cell below, print the logs of the web service and delete the service

### Deploy Webservice

In [18]:
from azureml.core.webservice import Webservice, AciWebservice
model = Model(ws,'Heart_Failure_Prediction_Model')

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 2, 
                                                       description='Heart_Failure_Prediction_Model',enable_app_insights=True )


service=Model.deploy(workspace=ws,
                    name="heart-failure-service",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config,overwrite=True)

service.wait_for_deployment(show_output=True)





Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-04-18 13:01:42+00:00 Creating Container Registry if not exists..
2022-04-18 13:11:42+00:00 Registering the environment..
2022-04-18 13:11:44+00:00 Building image..
2022-04-18 13:21:59+00:00 Generating deployment configuration.
2022-04-18 13:22:00+00:00 Submitting deployment to compute..
2022-04-18 13:22:04+00:00 Checking the status of deployment heart-failure-service..
2022-04-18 13:24:54+00:00 Checking the status of inference endpoint heart-failure-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [19]:
print(service.get_logs())

2022-04-18T13:24:26,700437600+00:00 - iot-server/run 
2022-04-18T13:24:26,717724800+00:00 - nginx/run 
2022-04-18T13:24:26,707210600+00:00 - gunicorn/run 
2022-04-18T13:24:26,708200000+00:00 - rsyslog/run 
Dynamic Python package installation is disabled.
Starting HTTP server
rsyslogd: /azureml-envs/azureml_4a0fd0e4e413efca88a642b691ad47f1/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-04-18T13:24:27,155656900+00:00 - iot-server/finish 1 0
2022-04-18T13:24:27,158143300+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (75)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 103
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2022-04-18 13:24:31,046 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found.

### Test Webservice

In [20]:
import json
import requests


test = {
  "data": [ {
      "age": 90,
      "anaemia": 0,
      "creatinine_phosphokinase": 500, 
      "diabetes":  1,
      "ejection_fraction": 150,
      "high_blood_pressure": 0,
      "platelets": 100000,
      "serum_creatinine": 2.75,
      "serum_sodium": 140,
      "sex": 1,
      "smoking": 0,
      "time": 500 },
      
      {"age": 45,
      "anaemia": 1,
      "creatinine_phosphokinase": 500, 
      "diabetes":  1,
      "ejection_fraction": 50,
      "high_blood_pressure": 1,
      "platelets": 100000,
      "serum_creatinine": 1.75,
      "serum_sodium": 140,
      "sex": 0,
      "smoking": 1,
      "time": 213}
  ]
}


test_data = json.dumps(test)

#Test response
response = requests.post(service.scoring_uri, test_data, headers={'Content-Type': 'application/json'})


print(response.json())


[0, 0]


### Delete Webservice

In [22]:
service.delete()

model.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
